<a href="https://colab.research.google.com/github/lfmartins/introduction-to-computational-mathematics/blob/main/16-stiffnessandhigherorderODEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stiff Problems

Many times when talking about stability of our methods, we talk about the stiffness of a system of equations. The formal definition of stiffness is a bit vague but generally the idea is asking how small of a timestep do we need to take to have a stable numerical solution.

For instance

$$y'= -y,\qquad t\in[a,b]$$

is a simple ODE that can give solutions that are not converging to the correct answer if the tolerances are set too high.


In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
def fun1(t,y):
    f=-1*y
    return f

sol0=solve_ivp(fun1,[0,10],[1],rtol=.01)
sol1=solve_ivp(fun1,[0,10],[1],rtol=.1)
sol2=solve_ivp(fun1,[0,10],[1],rtol=1)
sol3=solve_ivp(fun1,[0,10],[1],rtol=2)

plt.plot(sol0.t,sol0.y[0,:],sol1.t,sol1.y[0,:],sol2.t,sol2.y[0,:],sol3.t,sol3.y[0,:])

Generally, there is a rule that

$$y'=\lambda y,\qquad t\in[t_0,t_N]$$

required a small time step for Forward Euler generally if $(t_N-t_0)Re(\lambda)<<-1$. This is due to the rapid decay of the system. 

For systems
$$\mathbf{y}'=\mathbf{f}(\mathbf{y},t)$$
we can perform a similar analysis if we examine the Jacobian of the the system
$$J=\frac{\partial \mathbf{f}_i}{\partial \mathbf{y}_j}=\begin{bmatrix} {\partial f_1\over\partial y_{1}} & {\partial f_1\over\partial y_{2}}\\
{\partial f_2\over\partial y_{1}} & {\partial f_2\over\partial y_{2}}
\end{bmatrix}$$
where $i$ stands for the row of the Jacobian and $j$ stands for the column of the Jacobian. We can make the argument that a system is stiff if

$(t_N-t_0)\min_{j}(Re(\lambda_{j}))<<-1$

where $\lambda_{j}$ is an eigenvalue of the Jacobian. This can be found using eigen solve present in `numpy.linalg`.


### Example 
Let's check out the stiffness of the Lotka Volterra equations. Our first step is to calculate the Jacobian of the system.

We're going to first rewrite our variables $X$ and $Y$ as $y_1$ and $y_2$ to correspond to how we wrote out our stiff formulation

$$y_1'=ay_1-b y_1 y_2=f_1(t,y_1,y_2),\qquad y_1(0)=y_{1,0}$$
$$y_2'=c y_1 y_2 - dy_2=f_2(t,y_1,y_2),\qquad y_2(0)=y_{2,0}$$

where $a=1.5$, $b=1$, $c=0.1$, and $d=1$. Now let's take a partial derivative of $f_1$ and $f_2$ (i.e. take a derivative with respect to only the variable in the numerator of ${df_i\over dy_j}$ and treat all other variables as constants.

$${\partial f_1\over \partial y_1}=a-b y_2$$
$${\partial f_1\over \partial y_2}=-b y_1$$
$${\partial f_2\over \partial y_1}=c y_2$$
$${\partial f_2\over \partial y_2}=c y_1-d$$

So our Jacobian is

$$J=\begin{bmatrix} {\partial f_1\over\partial y_{1}} & {\partial f_1\over\partial y_{2}}\\
{\partial f_2\over\partial y_{1}} & {\partial f_2\over\partial y_{2}}
\end{bmatrix}=\begin{bmatrix} a-b y_2 & -b y_1\\
cy_2 & cy_1-d
\end{bmatrix}$$

plugging in the given values of $a,$ $b,$ $c,$ and $d$, and choosing $y_1=40$ and $y_2=.1$ to correspond roughly to the solution during a period where a lot of changes are occurring (i.e. when either the prey populations are very high and the predator population is very low). This value was taken from looking the previous solution. The resulting Jacobian with these values is:

$$J=\begin{bmatrix} 38.5 & -40\\
0.01 & 3
\end{bmatrix}.$$

Below is code to show how to calculate the eigenvalue of the Jacobian.

In [ ]:
import numpy as np
from numpy.linalg import eig

y1=40
y2=.1

a=1.5
b=1.
c=0.1
d=1.

J = np.array([[a-b*y1, -b*y1], 
              [c*y2, c*y1-d]])
print(J)
w,v=eig(J)
print('eigenvalue:', w)
print('eigenvector', v)

print('The smallest (real component) of the eigenvalues is:', np.min(np.real(w)))

# Higher Order ODEs

The **order of an ODE** is determined by **the highest order derivative present in the equation.**

So far we've been mostly dealing with **first-order ODEs**, where the highest order derivative present in the differential equation is of order 1, e.g. $dy/dt$, $dy/dx$, $y'$, etc.... 

However we did have one example that was a higher order ODE,

$$m{d^2y\over dt^2}+\beta{dy\over dt}+gy=0$$

This ODE is second order because the highest order derivative is ${d^2y\over dt^2}$. This is known as the spring-mass-damper equation.

Previously, we had to get an ODE into normal form to use our numerical methods
$${dy\over dt}=f(t,y) \qquad \mbox{or} \qquad y'=f(t,y)$$
For a higher order ODE, the highest order derivative is kept alone on the left-hand side and everything else is on the right-hand side. 
$$y''=f(t,y,y')$$
$$y'''=f(t,y,y',y'')$$
$$...$$
$$y^{(n)}=f(t,y',y'',...,y^{(n-1)})$$


We can set up normal form for  the spring-mass-damper equation
$$my''+\beta y'+gy=0,$$
first by isolating $y''$,
$$⇒ my''=-\beta y'-gy$$
then dividing by the constant $m$
$$\Rightarrow y''=-\frac{\beta}{m}y' -\frac{g}{m}y$$
so that we have a normal form
$$\Rightarrow y''=f(t,y,y')$$
where we see $f(t,y,y')=-\frac{\beta}{m}y' -\frac{g}{m}y$. 






Why are we doing this? We have methods that work for this normal form... but how do we deal with the second derivative? 

One trick is to treat $y$ and $y'$ as separate variables $y_1$ and $y_2$ respectively. With $y=y_1$ and $y'=y_2$, we can then say $y''=y'_2$

$$ y''=-\frac{\beta}{m}y' -\frac{g}{m}y$$
$$\Rightarrow  y_2'=-\frac{\beta}{m}y_2 -\frac{g}{m}y_1$$

We're still interested in a solution $y$, so we can update $y$ using the form 
$$y'_1=y_2$$, since this is equivalent to $y'=y'$

Combining these two parts, we turned our second order ODE into a system of first order ODES,

$$y'_1=y_2$$
$$y'_2=-\frac{\beta}{m}y_2 -\frac{g}{m}y_1$$

We can use the methods learned in Lesson 14 for systems of ODES using the following form,

$$y'_1=f_1(t,y_1,y_2)$$
$$y'_2=f_2(t,y_1,y_2).$$

Below we've chosen $m=1$, $\beta=0.1$, and $g=.1$, with initial conditions $y=y_1=1.0$ and $y'=y_2=0.0$


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def SMDEqn(t,Z):
    m=1
    beta=0.1
    g=.1
    y1,y2=Z
    f1=y2
    f2=-(beta/m)*y2-(g/m)*y1
    return [f1,f2]

sol = solve_ivp(SMDEqn, [0, 100], [1, 0],'RK45',rtol=1e-6)
plt.plot(sol.t,sol.y[0,:],'-vb')

Generally the behavior of the system depends on the value of $\beta^2-4mg$. In the previous example we had $\beta^2-4mg<0$. We call such a system underdamped, where the solution resembles $y=e^{p_1t}(A\cos(\omega t)+B\sin(\omega t))$.

###Exercises
1. Rerun with values such that $\beta^2-4mg>0$. Try also with initial conditions $y=1$ and $y'=0.1$. We call such a system overdamped, where the solution resembles $y=Ae^{p_1t}+Be^{p_2t}$.
2. Rerun with values such that $\beta^2-4mg=0$. Try also with initial conditions $y=1$ and $y'=0.1$. We call such a system critically damped, where the solution resembles $y=(A+Bt)e^{pt}$.
3. Rerun with the original values but $\beta=0$, where the solution resembles $y=A\cos(\omega t)+B\sin(\omega t)$.